In [1]:
from google import genai
from google.genai import types
import base64
import json
import os
from dotenv import load_dotenv

import unicodedata
import re

In [2]:
load_dotenv()
project_name = os.getenv('PROJECT_NAME')

The story id, from 1 to 13 according to the 13 Ainu Kamuy Yukars translated by Chiri Yukie. The Yukar ID starts at 1. Chiri's Preface is 0.

In [3]:
start_at = 1
end_at = 13

In [4]:
system_instruction_prompt = """You are a professional translator. You know Japanese, English and Chinese. You can translate Japanese into either Chinese or English."""

client = genai.Client(
      vertexai=True,
      project=project_name,
      location="us-central1",
)

model = "gemini-2.0-flash-001"

generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=system_instruction_prompt)],
  )

In [5]:
poetic_translation_prompt = "Translate the following text from Japanese to English. The original text is a poem. Try to make the translation poetic but keep the original meanings."

descriptive_translation_prompt = "Translate the following text from Japanese to English. Keep the original meanings."

In [6]:
# read the content page of Japanese translation and get the Japanese translated title
with open("Chiri_Japanese_Translation/content.txt", "r", encoding="utf8") as f:
    japanese_content = f.read()
    japanese_content = unicodedata.normalize('NFKC', japanese_content)


s=re.split(r'\n\n', japanese_content)
japanese_titles = re.split(r'\n', s[1])

In [7]:
# read the content page of Ainu original text and get the original title
with open("original_Ainu_text/content.txt", "r", encoding="utf8") as f:
    ainu_content = f.read()
    ainu_content = unicodedata.normalize('NFKC', ainu_content)


s=re.split(r'\n\n', ainu_content)
ainu_titles = re.split(r'\n', s[1])

In [8]:
# Read the markdown template for writing the Chinese translations to Markdown file

#read in the template
with open("templates/raw_output_md_template", "r", encoding="utf8") as f:
    md_template = f.read()
    md_template = unicodedata.normalize('NFKC', md_template)

In [9]:
def generate(client: genai.Client,generate_content_config :types.GenerateContentConfig,model :str, /,input_text :str, prompt :str):

    text_full_prompt = text1 = types.Part.from_text(text=f"{prompt}\n\n{input_text}")

    output = ""

    contents = [
      types.Content(
        role="user",
        parts=[
          text_full_prompt
        ]
      )
    ]

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        output += chunk.text

    return output

def get_output_file_name_key(title :str):
    # setup the output file name
    s = title.split()
    md_name_part = s[0]

    name_2nd_part = ""

    for text in s:
        if text.startswith('“'):
            name_2nd_part = text.replace('“', '').replace('”', '')
        
    md_name_part += "_" + name_2nd_part

    return md_name_part



In [ ]:
# process the translation in batch mode

for song_no in range(start_at, end_at + 1):
    with open(f"Chiri_Japanese_Translation/story_translation_{song_no}.txt", "r", encoding="utf8") as f:
        japanese_story = f.read()
        japanese_story = unicodedata.normalize('NFKC', japanese_story)

        chinese_poetic_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=poetic_translation_prompt)

        chinese_descriptive_translation = generate(client,generate_content_config,model,input_text=japanese_story,prompt=descriptive_translation_prompt)

        md_output = md_template.format(translated_language="English", ainu_title=ainu_titles[song_no - 1],
                               poetic_prompt=poetic_translation_prompt, descriptive_prompt=descriptive_translation_prompt,
                               japanese_title=japanese_titles[song_no], input_japanese = japanese_story,
                               output_poetic=chinese_poetic_translation, output_descriptive=chinese_descriptive_translation)
        
        md_name_part = get_output_file_name_key(ainu_titles[song_no - 1])

        with open(f"LLM_prompts_and_raw_outputs/English_Translation/{song_no}_{md_name_part}_to_English.md", "w", encoding="utf8") as f:
            f.write(md_output)



Here's a translation of the poem, aiming for a balance between accuracy and a poetic feel:

**The Owl God's Own Song**

"Silver drops fall, fall around,
Golden drops fall, fall around."

Singing this song, I drift along the stream,
Looking down on human villages below.
The poor of yesterday are rich today, it seems,
The rich of yesterday, now bowed in woe.

By the shore, children play with toy bows and arrows,
"Silver drops fall, fall around,
Golden drops fall, fall around."
Singing, I pass above them, shadows.

They run beneath me, shouting in glee,
"A beautiful bird! A god's own bird!
Come, shoot an arrow! He who strikes it, see,
The god's own bird, is bravest, strongest, heard!"

The children of the newly rich, with golden bows,
And golden arrows, aim and let fly.
I weave and dodge, above, below,
Their gilded shafts against the sky.

Among them, one child, with a simple, wooden bow,
And simple arrow, joins the fray.
A pauper's child, his clothes betray,
Yet in his eyes, a noble fire